In [8]:
import logging

logging.basicConfig(level=logging.INFO)

In [9]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss, MSELoss

# from pennylane import NesterovMomentumOptimizer

from qcc.ml.data import Data, BinaryData
from qcc.ml.optimize import Optimizer
from qcc.ml.data import ImageTransform, ImageTransform1D
from qcc.experiment import Experiment
from qcc.ml.model import Model

from pathlib import Path
from qcc.file import new_dir

from qcc.quantum.operation.ansatz import ConvolutionPoolingAnsatz as Module
# from qcc.quantum.operation.ansatz import BaselineAnsatz as Ansatz

# from qcc.quantum.operation.ansatz.basic import BasicFiltering6
from qcc.quantum.operation.fully_connected import FullyConnected

In [10]:
# Meta parameters
name = "test"
path = Path(f"results/{name}")
num_trials = 1
silent = False
is_quantum = False

# Ansatz parameters
dims = (16, 16)
num_layers = 1
module_options = {
    # "U_filter": BasicFiltering6,
    # "U_fully_connected": None,
    # "pooling": True,
    # "filter_shape": (2, 2, 2)
}

In [ ]:
path = new_dir(path, overwrite=True)
if is_quantum:
    module = Module.from_dims
else:
    module = Module

# Create module
module: module(
    dims,
    num_layers=num_layers,
    **module_options,
)
data = Data(
    FashionMNIST,
    ImageTransform(dims)
    batch_size=(8,1000),
)
optimizer = Optimizer(Adam)
loss = CrossEntropyLoss()
model = Model.with_logging(module, data, optimizer, loss, epoch=1)

# Log important values
model.logger.info(f"Circuit ID: {name}")
model.logger.info(f"{module=}")
model.logger.info(f"{data=}")
model.logger.info(f"{optimizer=}")
model.logger.info(f"{loss=}")

model.logger.info(f"{num_trials=}")
model.logger.info(f"{dims=}")
model.logger.info(f"{num_layers=}")
# model.logger.info(f"{1=}")
model.logger.info(f"{module_options=}")

# Save circuit drawing
if is_quantum:
    filename = path / f"{name}_circuit.png"
    fig = module.draw(filename=filename, decompose=True)
    display(fig)

In [ ]:
# Run experiment
results_schema = ["accuracy", "training_time", "testing_time"]
experiment = Experiment(model, num_trials, results_schema)
experiment.partial(silent=silent)
results = experiment(filename=path / name)

# Print accuracy results
metrics = ("median", "mean", "max", "min", "std")
for name in results.columns:
    col = results[name]
    msg = (f"{metric}={getattr(col, metric)():.05}" for metric in metrics)
    msg = ", ".join(msg)
    msg = f"{name}: {msg}"
    model.logger.info(msg)

# Save aggregated loss history figure
display(experiment.draw(path / f"{name}.png"))